In [9]:
import pandas as pd

In [28]:
def table_to_network(df_in, c):
    df = df_in.copy()
    df = df.groupby(c).apply(lambda x: x.iloc[:15]).reset_index(drop=True)
    nodes = pd.concat([df[c].drop_duplicates(), df.entity.drop_duplicates()], ignore_index=True)
    nodes = nodes.reset_index().rename(columns={0: 'label', 'index': 'id'})
    num_persons = len(df[c].drop_duplicates())
    nodes['group'] = ([c]*num_persons)+(['word']*(nodes.shape[0]-num_persons))
    nodes['label'] = nodes['label'].astype(str)
    
    edges = df[[c,'entity']]
    edges[c] = edges[c].astype(str)

    edges.columns = ['from','to']
    edges = edges.merge(nodes, left_on='from', right_on='label').iloc[:,:-2]\
                 .merge(nodes, left_on='to', right_on='label').iloc[:,:-2]

    edges = edges.iloc[:, 2:]
    edges.columns = ['from','to']
    
    return nodes, edges

In [35]:
nodes, edges = table_to_network(pd.read_excel('ents_by_monthyear.xlsx'),'MonthYear_')

/home/ray/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [36]:
nodes.to_json('nodes_monthyear.json', orient='records')

edges.to_json('edges_monthyear.json', orient='records')